In [ ]:
-- 1. Create Node Tables

-- Emperors Node
CREATE TABLE EmperorNode
(
    EmperorID INT,
    RegnalName NVARCHAR(100),
    BirthName NVARCHAR(100),
    BirthDate DATE,
    DeathDate DATE,
    PRIMARY KEY (EmperorID)
) AS NODE;

-- Dynasty Node
CREATE TABLE DynastyNode
(
    DynastyID INT,
    DynastyName NVARCHAR(100),
    StartYear INT,
    EndYear INT,
    PRIMARY KEY (DynastyID)
) AS NODE;

-- Province Node
CREATE TABLE ProvinceNode
(
    ProvinceID INT,
    ProvinceName NVARCHAR(100),
    Region NVARCHAR(100),
    EstablishedDate DATE,
    PRIMARY KEY (ProvinceID)
) AS NODE;

-- Battle Node
CREATE TABLE BattleNode
(
    BattleID INT,
    BattleName NVARCHAR(100),
    BattleDate DATE,
    Location NVARCHAR(100),
    Outcome NVARCHAR(50),
    PRIMARY KEY (BattleID)
) AS NODE;

-- 2. Create Edge Tables

-- Succession Edge (Emperor to Emperor)
CREATE TABLE SucceededBy AS EDGE;

-- Dynasty Membership Edge
CREATE TABLE BelongsToDynasty AS EDGE;

-- Province Governance Edge
CREATE TABLE Governed AS EDGE;

-- Battle Participation Edge
CREATE TABLE ParticipatedIn 
(
    Role NVARCHAR(50),  -- e.g., 'Commander', 'Victor', 'Defeated'
    Description NVARCHAR(MAX)
) AS EDGE;

-- Family Relationship Edge
CREATE TABLE FamilyRelation
(
    RelationType NVARCHAR(50),  -- e.g., 'Father', 'Son', 'Adopted'
    RelationDate DATE
) AS EDGE;

-- 3. Insert Sample Data

-- Insert Nodes
INSERT INTO EmperorNode
VALUES 
(1, 'Augustus', 'Gaius Octavius', '63-09-23 BC', '14-08-19 AD'),
(2, 'Tiberius', 'Tiberius Claudius Nero', '42-11-16 BC', '37-03-16 AD'),
(3, 'Caligula', 'Gaius Julius Caesar Germanicus', '12-08-31 AD', '41-01-24 AD');

INSERT INTO DynastyNode
VALUES 
(1, 'Julio-Claudian', -27, 68);

INSERT INTO ProvinceNode
VALUES 
(1, 'Egypt', 'Africa', '30 BC'),
(2, 'Gaul', 'Europe', '50 BC');

INSERT INTO BattleNode
VALUES 
(1, 'Battle of Actium', '31-09-02 BC', 'Actium', 'Victory');

-- Insert Edges
-- Succession relationships
INSERT INTO SucceededBy
VALUES (
    (SELECT $node_id FROM EmperorNode WHERE EmperorID = 1),
    (SELECT $node_id FROM EmperorNode WHERE EmperorID = 2)
);

-- Dynasty membership
INSERT INTO BelongsToDynasty
VALUES (
    (SELECT $node_id FROM EmperorNode WHERE EmperorID = 1),
    (SELECT $node_id FROM DynastyNode WHERE DynastyID = 1)
);

-- Province governance
INSERT INTO Governed
VALUES (
    (SELECT $node_id FROM EmperorNode WHERE EmperorID = 1),
    (SELECT $node_id FROM ProvinceNode WHERE ProvinceID = 1)
);

-- Battle participation
INSERT INTO ParticipatedIn
VALUES (
    (SELECT $node_id FROM EmperorNode WHERE EmperorID = 1),
    (SELECT $node_id FROM BattleNode WHERE BattleID = 1),
    'Commander',
    'Led naval forces to victory'
);

-- 4. Complex Graph Queries

-- Find succession chain
SELECT 
    e1.RegnalName AS Emperor,
    e2.RegnalName AS Successor,
    e2.BirthDate
FROM 
    EmperorNode e1,
    SucceededBy,
    EmperorNode e2
WHERE MATCH(e1-(SucceededBy)->e2);

-- Find all emperors of a dynasty with their provinces
SELECT 
    e.RegnalName,
    d.DynastyName,
    p.ProvinceName
FROM 
    EmperorNode e,
    BelongsToDynasty,
    DynastyNode d,
    Governed,
    ProvinceNode p
WHERE MATCH(e-(BelongsToDynasty)->d AND e-(Governed)->p);

-- Find battles and participating emperors
SELECT 
    e.RegnalName,
    b.BattleName,
    p.Role,
    p.Description
FROM 
    EmperorNode e,
    ParticipatedIn p,
    BattleNode b
WHERE MATCH(e-(ParticipatedIn)->b);

-- 5. Advanced Graph Queries

-- Find shortest path between two emperors
WITH ShortestPath AS (
    SELECT
        e1.RegnalName AS StartEmperor,
        e2.RegnalName AS EndEmperor,
        STRING_AGG(m.RegnalName, ' -> ') WITHIN GROUP (GRAPH PATH) AS Path,
        LAST_VALUE(m.RegnalName) WITHIN GROUP (GRAPH PATH) AS LastInPath,
        COUNT(m.RegnalName) WITHIN GROUP (GRAPH PATH) AS PathLength
    FROM
        EmperorNode e1,
        EmperorNode e2,
        EmperorNode m
    WHERE MATCH(SHORTEST_PATH(e1(-(SucceededBy)->m)+ ->e2))
        AND e1.EmperorID = 1 AND e2.EmperorID = 3
)
SELECT * FROM ShortestPath;

-- Find all emperors within 2 degrees of separation
SELECT DISTINCT
    e1.RegnalName AS Emperor,
    e2.RegnalName AS RelatedEmperor,
    COUNT(e2.RegnalName) WITHIN GROUP (GRAPH PATH) AS Degree
FROM
    EmperorNode e1,
    EmperorNode e2
WHERE MATCH(SHORTEST_PATH(e1(-(FamilyRelation|SucceededBy)->)+ ->e2))
    AND e1.EmperorID = 1
    AND COUNT(e2.RegnalName) WITHIN GROUP (GRAPH PATH) <= 2;

-- 6. Analytical Queries

-- Find most connected emperors
SELECT 
    e.RegnalName,
    COUNT(p1.ProvinceName) as ProvinceCount,
    COUNT(b1.BattleName) as BattleCount
FROM 
    EmperorNode e
    LEFT JOIN (Governed, ProvinceNode p1) ON MATCH(e-(Governed)->p1)
    LEFT JOIN (ParticipatedIn, BattleNode b1) ON MATCH(e-(ParticipatedIn)->b1)
GROUP BY e.RegnalName
ORDER BY (COUNT(p1.ProvinceName) + COUNT(b1.BattleName)) DESC;

-- Find dynasty influence over time
SELECT 
    d.DynastyName,
    YEAR(e.BirthDate) as Year,
    COUNT(DISTINCT p.ProvinceID) as ProvincesControlled
FROM 
    DynastyNode d,
    BelongsToDynasty,
    EmperorNode e,
    Governed,
    ProvinceNode p
WHERE MATCH(d<-(BelongsToDynasty)-e-(Governed)->p)
GROUP BY d.DynastyName, YEAR(e.BirthDate)
ORDER BY Year;

-- 7. Maintenance and Optimization

-- Create indexes for better performance
CREATE INDEX IX_Emperor_RegnalName ON EmperorNode(RegnalName);
CREATE INDEX IX_Dynasty_Name ON DynastyNode(DynastyName);
CREATE INDEX IX_Province_Name ON ProvinceNode(ProvinceName);
CREATE INDEX IX_Battle_Date ON BattleNode(BattleDate);

-- Create procedure to add new succession relationship
CREATE OR ALTER PROCEDURE AddSuccession
    @PredecessorID INT,
    @SuccessorID INT
AS
BEGIN
    INSERT INTO SucceededBy
    VALUES (
        (SELECT $node_id FROM EmperorNode WHERE EmperorID = @PredecessorID),
        (SELECT $node_id FROM EmperorNode WHERE EmperorID = @SuccessorID)
    );
END;
GO

-- Create procedure to analyze emperor's influence
CREATE OR ALTER PROCEDURE AnalyzeEmperorInfluence
    @EmperorID INT
AS
BEGIN
    SELECT 
        e.RegnalName,
        COUNT(DISTINCT p.ProvinceID) as ProvinceCount,
        COUNT(DISTINCT b.BattleID) as BattleCount,
        COUNT(DISTINCT s.EmperorID) as SuccessorCount
    FROM 
        EmperorNode e
        LEFT JOIN (Governed, ProvinceNode p) ON MATCH(e-(Governed)->p)
        LEFT JOIN (ParticipatedIn, BattleNode b) ON MATCH(e-(ParticipatedIn)->b)
        LEFT JOIN (SucceededBy, EmperorNode s) ON MATCH(e-(SucceededBy)->s)
    WHERE e.EmperorID = @EmperorID
    GROUP BY e.RegnalName;
END;
GO

In [ ]:
-- 1. Graph Integrity Check Procedures
CREATE OR ALTER PROCEDURE ValidateGraphIntegrity
AS
BEGIN
    SET NOCOUNT ON;
    
    CREATE TABLE #ValidationResults
    (
        CheckID INT,
        CheckName NVARCHAR(100),
        Issue NVARCHAR(MAX),
        AffectedNodes INT,
        Severity INT -- 1: Info, 2: Warning, 3: Critical
    );

    -- Check for orphaned nodes (emperors without dynasty)
    INSERT INTO #ValidationResults
    SELECT 
        1,
        'Orphaned Emperors',
        'Emperors without dynasty connection',
        COUNT(*),
        2
    FROM EmperorNode e
    WHERE NOT EXISTS (
        SELECT 1 
        FROM BelongsToDynasty b 
        WHERE e.$node_id = b.$from_id
    );

    -- Check for broken succession chains
    INSERT INTO #ValidationResults
    SELECT 
        2,
        'Broken Succession',
        'Gaps in succession chain',
        COUNT(*),
        3
    FROM EmperorNode e1
    JOIN SucceededBy s ON e1.$node_id = s.$from_id
    LEFT JOIN EmperorNode e2 ON s.$to_id = e2.$node_id
    WHERE e2.$node_id IS NULL;

    -- Check for circular references
    INSERT INTO #ValidationResults
    SELECT 
        3,
        'Circular Reference',
        'Detected circular succession',
        COUNT(*),
        3
    FROM EmperorNode e1
    JOIN SucceededBy s1 ON e1.$node_id = s1.$from_id
    JOIN SucceededBy s2 ON s1.$to_id = s2.$from_id
    WHERE s2.$to_id = e1.$node_id;

    -- Return results
    SELECT * FROM #ValidationResults ORDER BY Severity DESC;
    DROP TABLE #ValidationResults;
END;
GO

-- 2. Graph Statistics and Health Check
CREATE OR ALTER PROCEDURE AnalyzeGraphHealth
AS
BEGIN
    SET NOCOUNT ON;

    -- Node statistics
    SELECT 
        'Node Counts' as Metric,
        (SELECT COUNT(*) FROM EmperorNode) as Emperors,
        (SELECT COUNT(*) FROM DynastyNode) as Dynasties,
        (SELECT COUNT(*) FROM ProvinceNode) as Provinces,
        (SELECT COUNT(*) FROM BattleNode) as Battles;

    -- Edge statistics
    SELECT 
        'Edge Counts' as Metric,
        (SELECT COUNT(*) FROM SucceededBy) as Successions,
        (SELECT COUNT(*) FROM BelongsToDynasty) as DynastyMemberships,
        (SELECT COUNT(*) FROM Governed) as Governances,
        (SELECT COUNT(*) FROM ParticipatedIn) as BattleParticipations;

    -- Node connectivity analysis
    SELECT 
        'Connectivity' as Metric,
        AVG(Connections) as AvgConnections,
        MAX(Connections) as MaxConnections
    FROM (
        SELECT e.EmperorID, COUNT(*) as Connections
        FROM EmperorNode e
        LEFT JOIN (
            SELECT $from_id FROM SucceededBy
            UNION ALL
            SELECT $from_id FROM BelongsToDynasty
            UNION ALL
            SELECT $from_id FROM Governed
            UNION ALL
            SELECT $from_id FROM ParticipatedIn
        ) as Edges ON e.$node_id = Edges.$from_id
        GROUP BY e.EmperorID
    ) as ConnectionCounts;
END;
GO

-- 3. Graph Maintenance and Cleanup
CREATE OR ALTER PROCEDURE CleanupGraphData
AS
BEGIN
    SET NOCOUNT ON;
    BEGIN TRANSACTION;
    
    BEGIN TRY
        -- Remove invalid edges
        DELETE FROM SucceededBy 
        WHERE $from_id NOT IN (SELECT $node_id FROM EmperorNode)
           OR $to_id NOT IN (SELECT $node_id FROM EmperorNode);

        DELETE FROM BelongsToDynasty
        WHERE $from_id NOT IN (SELECT $node_id FROM EmperorNode)
           OR $to_id NOT IN (SELECT $node_id FROM DynastyNode);

        -- Remove duplicate edges
        WITH DuplicateEdges AS (
            SELECT 
                $from_id, 
                $to_id,
                ROW_NUMBER() OVER (PARTITION BY $from_id, $to_id ORDER BY $edge_id) as RowNum
            FROM SucceededBy
        )
        DELETE FROM DuplicateEdges WHERE RowNum > 1;

        COMMIT;
        SELECT 'Cleanup completed successfully' as Status;
    END TRY
    BEGIN CATCH
        ROLLBACK;
        SELECT 
            ERROR_NUMBER() as ErrorNumber,
            ERROR_MESSAGE() as ErrorMessage;
    END CATCH;
END;
GO

-- 4. Graph Index Maintenance
CREATE OR ALTER PROCEDURE RebuildGraphIndexes
AS
BEGIN
    SET NOCOUNT ON;
    
    DECLARE @SQL NVARCHAR(MAX) = '';
    
    -- Get all indexes on graph tables
    SELECT @SQL = @SQL + 
        'ALTER INDEX ' + i.name + ' ON ' + OBJECT_NAME(i.object_id) + 
        ' REBUILD WITH (ONLINE = ON);' + CHAR(13)
    FROM sys.indexes i
    JOIN sys.tables t ON i.object_id = t.object_id
    WHERE t.is_node = 1 OR t.is_edge = 1;
    
    BEGIN TRY
        EXEC sp_executesql @SQL;
        SELECT 'Index rebuild completed successfully' as Status;
    END TRY
    BEGIN CATCH
        SELECT 
            ERROR_NUMBER() as ErrorNumber,
            ERROR_MESSAGE() as ErrorMessage;
    END CATCH;
END;
GO

-- 5. Graph Performance Monitoring
CREATE OR ALTER PROCEDURE MonitorGraphPerformance
AS
BEGIN
    SET NOCOUNT ON;

    -- Query performance statistics
    SELECT TOP 10
        qs.execution_count,
        qs.total_elapsed_time/qs.execution_count as avg_elapsed_time,
        qs.total_logical_reads/qs.execution_count as avg_logical_reads,
        SUBSTRING(qt.text, (qs.statement_start_offset/2)+1,
            ((CASE qs.statement_end_offset
                WHEN -1 THEN DATALENGTH(qt.text)
                ELSE qs.statement_end_offset
                END - qs.statement_start_offset)/2) + 1) as query_text
    FROM sys.dm_exec_query_stats qs
    CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
    WHERE qt.text LIKE '%MATCH%'
    ORDER BY avg_elapsed_time DESC;

    -- Index usage statistics
    SELECT 
        OBJECT_NAME(i.object_id) as TableName,
        i.name as IndexName,
        ius.user_seeks,
        ius.user_scans,
        ius.user_lookups,
        ius.user_updates
    FROM sys.dm_db_index_usage_stats ius
    JOIN sys.indexes i ON ius.object_id = i.object_id
        AND ius.index_id = i.index_id
    JOIN sys.tables t ON i.object_id = t.object_id
    WHERE t.is_node = 1 OR t.is_edge = 1;
END;
GO

-- 6. Graph Archival Procedure
CREATE OR ALTER PROCEDURE ArchiveGraphData
    @YearThreshold INT
AS
BEGIN
    SET NOCOUNT ON;
    BEGIN TRANSACTION;

    BEGIN TRY
        -- Create archive tables if they don't exist
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'ArchivedEmperorNode')
        BEGIN
            CREATE TABLE ArchivedEmperorNode AS NODE AS SELECT * FROM EmperorNode WHERE 1=0;
            CREATE TABLE ArchivedSucceededBy AS EDGE AS SELECT * FROM SucceededBy WHERE 1=0;
        END;

        -- Move old data to archive
        INSERT INTO ArchivedEmperorNode
        SELECT * FROM EmperorNode
        WHERE YEAR(DeathDate) < @YearThreshold;

        INSERT INTO ArchivedSucceededBy
        SELECT s.*
        FROM SucceededBy s
        JOIN EmperorNode e ON s.$from_id = e.$node_id
        WHERE YEAR(e.DeathDate) < @YearThreshold;

        -- Delete archived data from main tables
        DELETE FROM SucceededBy
        WHERE $from_id IN (
            SELECT $node_id 
            FROM EmperorNode 
            WHERE YEAR(DeathDate) < @YearThreshold
        );

        DELETE FROM EmperorNode
        WHERE YEAR(DeathDate) < @YearThreshold;

        COMMIT;
        SELECT 'Archival completed successfully' as Status;
    END TRY
    BEGIN CATCH
        ROLLBACK;
        SELECT 
            ERROR_NUMBER() as ErrorNumber,
            ERROR_MESSAGE() as ErrorMessage;
    END CATCH;
END;
GO

-- 7. Usage Example
-- Execute maintenance procedures in sequence
CREATE OR ALTER PROCEDURE ExecuteFullMaintenance
AS
BEGIN
    -- 1. Check graph integrity
    EXEC ValidateGraphIntegrity;
    
    -- 2. Analyze graph health
    EXEC AnalyzeGraphHealth;
    
    -- 3. Clean up invalid data
    EXEC CleanupGraphData;
    
    -- 4. Rebuild indexes
    EXEC RebuildGraphIndexes;
    
    -- 5. Monitor performance
    EXEC MonitorGraphPerformance;
    
    -- 6. Archive old data (example threshold)
    EXEC ArchiveGraphData @YearThreshold = 0;  -- Year 0 AD
END;
GO

In [ ]:
-- 1. Backup Procedures

-- Full backup with pre-checks
CREATE OR ALTER PROCEDURE ExecuteFullGraphBackup
    @BackupPath NVARCHAR(1000),
    @DatabaseName NVARCHAR(100)
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Validate backup path
    IF NOT EXISTS (
        SELECT 1 FROM sys.dm_os_volume_stats(DB_ID(), 1)
        WHERE volume_mount_point = LEFT(@BackupPath, 1) + ':\'
    )
    BEGIN
        RAISERROR('Invalid backup path specified.', 16, 1);
        RETURN;
    END;

    -- Pre-backup integrity check
    DBCC CHECKDB (@DatabaseName) WITH NO_INFOMSGS;

    -- Backup filename with timestamp
    DECLARE @BackupFile NVARCHAR(1000) = @BackupPath + 
        @DatabaseName + '_' + 
        REPLACE(CONVERT(NVARCHAR(20), GETDATE(), 120), ':', '') + 
        '.bak';

    -- Execute full backup
    BACKUP DATABASE @DatabaseName 
    TO DISK = @BackupFile
    WITH 
        COMPRESSION,
        CHECKSUM,
        STATS = 10,
        NAME = 'Full Graph Database Backup';

    -- Log backup details
    INSERT INTO BackupHistory (
        BackupType,
        BackupFile,
        BackupStart,
        BackupEnd,
        Status
    )
    VALUES (
        'FULL',
        @BackupFile,
        GETDATE(),
        GETDATE(),
        'Completed'
    );
END;
GO

-- Differential backup
CREATE OR ALTER PROCEDURE ExecuteDiffGraphBackup
    @BackupPath NVARCHAR(1000),
    @DatabaseName NVARCHAR(100)
AS
BEGIN
    SET NOCOUNT ON;

    -- Check if full backup exists
    IF NOT EXISTS (
        SELECT 1 
        FROM msdb.dbo.backupset 
        WHERE database_name = @DatabaseName 
        AND type = 'D'
    )
    BEGIN
        RAISERROR('No full backup found. Please perform full backup first.', 16, 1);
        RETURN;
    END;

    -- Backup filename
    DECLARE @BackupFile NVARCHAR(1000) = @BackupPath + 
        @DatabaseName + '_DIFF_' + 
        REPLACE(CONVERT(NVARCHAR(20), GETDATE(), 120), ':', '') + 
        '.bak';

    -- Execute differential backup
    BACKUP DATABASE @DatabaseName 
    TO DISK = @BackupFile
    WITH 
        DIFFERENTIAL,
        COMPRESSION,
        CHECKSUM,
        STATS = 10,
        NAME = 'Differential Graph Database Backup';
END;
GO

-- Transaction log backup
CREATE OR ALTER PROCEDURE ExecuteLogGraphBackup
    @BackupPath NVARCHAR(1000),
    @DatabaseName NVARCHAR(100)
AS
BEGIN
    SET NOCOUNT ON;

    -- Check recovery model
    IF (SELECT recovery_model_desc 
        FROM sys.databases 
        WHERE name = @DatabaseName) != 'FULL'
    BEGIN
        RAISERROR('Database must be in FULL recovery model for log backups.', 16, 1);
        RETURN;
    END;

    -- Backup filename
    DECLARE @BackupFile NVARCHAR(1000) = @BackupPath + 
        @DatabaseName + '_LOG_' + 
        REPLACE(CONVERT(NVARCHAR(20), GETDATE(), 120), ':', '') + 
        '.trn';

    -- Execute log backup
    BACKUP LOG @DatabaseName 
    TO DISK = @BackupFile
    WITH 
        COMPRESSION,
        CHECKSUM,
        STATS = 10,
        NAME = 'Transaction Log Backup';
END;
GO

-- 2. Recovery Procedures

-- Full database recovery
CREATE OR ALTER PROCEDURE dbo.RecoverGraphDatabase
    @BackupFile NVARCHAR(1000),
    @DatabaseName NVARCHAR(100),
    @NewDatabaseName NVARCHAR(100) = NULL
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Use new database name if specified
    DECLARE @RestoreDatabaseName NVARCHAR(100) = 
        ISNULL(@NewDatabaseName, @DatabaseName);

    -- Get logical file names from backup
    DECLARE @LogicalDataName NVARCHAR(128),
            @LogicalLogName NVARCHAR(128);

    SELECT 
        @LogicalDataName = logical_name
    FROM msdb.dbo.backupset b
    JOIN msdb.dbo.backupmediafamily m ON b.media_set_id = m.media_set_id
    JOIN msdb.dbo.backupfile f ON b.backup_set_id = f.backup_set_id
    WHERE b.database_name = @DatabaseName
    AND f.file_type = 'D';

    SELECT 
        @LogicalLogName = logical_name
    FROM msdb.dbo.backupset b
    JOIN msdb.dbo.backupmediafamily m ON b.media_set_id = m.media_set_id
    JOIN msdb.dbo.backupfile f ON b.backup_set_id = f.backup_set_id
    WHERE b.database_name = @DatabaseName
    AND f.file_type = 'L';

    -- Create the full paths
    DECLARE @DataFile NVARCHAR(1000) = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\' + @RestoreDatabaseName + '.mdf';
    DECLARE @LogFile NVARCHAR(1000) = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\' + @RestoreDatabaseName + '_log.ldf';

    -- Execute restore
    RESTORE DATABASE @RestoreDatabaseName
    FROM DISK = @BackupFile
    WITH 
        MOVE @LogicalDataName TO @DataFile,
        MOVE @LogicalLogName TO @LogFile,
        REPLACE,
        STATS = 10;
END;
GO

-- Point-in-time recovery
CREATE OR ALTER PROCEDURE RecoverGraphDatabaseToPoint
    @DatabaseName NVARCHAR(100),
    @TargetTime DATETIME,
    @NewDatabaseName NVARCHAR(100) = NULL
AS
BEGIN
    SET NOCOUNT ON;

    -- Find the last full backup before target time
    DECLARE @FullBackupFile NVARCHAR(1000);
    SELECT TOP 1 @FullBackupFile = physical_device_name
    FROM msdb.dbo.backupset b
    JOIN msdb.dbo.backupmediafamily m ON b.media_set_id = m.media_set_id
    WHERE database_name = @DatabaseName
    AND type = 'D'
    AND backup_start_date <= @TargetTime
    ORDER BY backup_start_date DESC;

    -- Restore full backup
    EXEC RecoverGraphDatabase 
        @BackupFile = @FullBackupFile,
        @DatabaseName = @DatabaseName,
        @NewDatabaseName = @NewDatabaseName;

    -- Find and apply relevant log backups
    DECLARE @LogBackupFile NVARCHAR(1000);
    DECLARE LogBackups CURSOR FOR
        SELECT physical_device_name
        FROM msdb.dbo.backupset b
        JOIN msdb.dbo.backupmediafamily m ON b.media_set_id = m.media_set_id
        WHERE database_name = @DatabaseName
        AND type = 'L'
        AND backup_start_date > (
            SELECT backup_start_date
            FROM msdb.dbo.backupset
            WHERE backup_set_id = (
                SELECT backup_set_id
                FROM msdb.dbo.backupmediafamily
                WHERE physical_device_name = @FullBackupFile
            )
        )
        AND backup_start_date <= @TargetTime
        ORDER BY backup_start_date;

    OPEN LogBackups;
    FETCH NEXT FROM LogBackups INTO @LogBackupFile;

    WHILE @@FETCH_STATUS = 0
    BEGIN
        RESTORE LOG @NewDatabaseName
        FROM DISK = @LogBackupFile
        WITH NORECOVERY, STOPAT = @TargetTime;

        FETCH NEXT FROM LogBackups INTO @LogBackupFile;
    END;

    CLOSE LogBackups;
    DEALLOCATE LogBackups;

    -- Recover the database
    RESTORE DATABASE @NewDatabaseName WITH RECOVERY;
END;
GO

-- 3. Verification Procedures

-- Verify backup integrity
CREATE OR ALTER PROCEDURE VerifyGraphBackup
    @BackupFile NVARCHAR(1000)
AS
BEGIN
    SET NOCOUNT ON;

    -- Verify backup
    RESTORE VERIFYONLY
    FROM DISK = @BackupFile
    WITH CHECKSUM;

    -- Check backup header
    RESTORE HEADERONLY
    FROM DISK = @BackupFile;

    -- Check backup file list
    RESTORE FILELISTONLY
    FROM DISK = @BackupFile;
END;
GO

-- Verify graph integrity after restore
CREATE OR ALTER PROCEDURE VerifyGraphIntegrityAfterRestore
    @DatabaseName NVARCHAR(100)
AS
BEGIN
    SET NOCOUNT ON;

    -- Declare variables for dynamic SQL
    DECLARE @SQL NVARCHAR(MAX)
    DECLARE @CheckDB NVARCHAR(MAX)
    
    -- Build CHECKDB command
    SET @CheckDB = 'DBCC CHECKDB (' + QUOTENAME(@DatabaseName) + ') WITH ALL_ERRORMSGS;'
    EXEC sp_executesql @CheckDB;

    -- Build query for orphaned edges check
    SET @SQL = '
    SELECT ''Orphaned Edges'' as Check_Type, COUNT(*) as Issue_Count
    FROM ' + QUOTENAME(@DatabaseName) + '.dbo.SucceededBy s
    WHERE NOT EXISTS (
        SELECT 1 FROM ' + QUOTENAME(@DatabaseName) + '.dbo.EmperorNode 
        WHERE $node_id = s.$from_id
    )
    OR NOT EXISTS (
        SELECT 1 FROM ' + QUOTENAME(@DatabaseName) + '.dbo.EmperorNode 
        WHERE $node_id = s.$to_id
    );

    SELECT ''Broken Relationships'' as Check_Type, COUNT(*) as Issue_Count
    FROM ' + QUOTENAME(@DatabaseName) + '.dbo.EmperorNode e
    LEFT JOIN ' + QUOTENAME(@DatabaseName) + '.dbo.SucceededBy s 
        ON e.$node_id = s.$from_id
    WHERE s.$edge_id IS NULL;'

    -- Execute the dynamic SQL
    EXEC sp_executesql @SQL;
END;
GO

-- 4. Backup Maintenance

-- Clean up old backups
CREATE OR ALTER PROCEDURE CleanupOldBackups
    @BackupPath NVARCHAR(1000),
    @RetentionDays INT = 30
AS
BEGIN
    SET NOCOUNT ON;

    -- Delete old backup files
    DECLARE @cmd NVARCHAR(1000);
    SET @cmd = 'forfiles /p "' + @BackupPath + 
               '" /s /m *.* /d -' + 
               CAST(@RetentionDays AS NVARCHAR(10)) + 
               ' /c "cmd /c del @path"';

    EXEC xp_cmdshell @cmd;

    -- Clean up backup history
    DECLARE @DeleteDate DATETIME = DATEADD(dd, -@RetentionDays, GETDATE());

    EXEC msdb.dbo.sp_delete_backuphistory @DeleteDate;
END;
GO

-- 5. Example Usage

-- Execute full backup routine
EXEC ExecuteFullGraphBackup 
    @BackupPath = 'C:\Backup\',
    @DatabaseName = 'RomanEmpireGraph';

-- Execute point-in-time recovery
EXEC RecoverGraphDatabaseToPoint
    @DatabaseName = 'RomanEmpireGraph',
    @TargetTime = '2024-02-10 12:00:00',
    @NewDatabaseName = 'RomanEmpireGraph_Recovered';

-- Verify recovery
EXEC VerifyGraphIntegrityAfterRestore
    @DatabaseName = 'RomanEmpireGraph_Recovered';

In [ ]:
CREATE OR ALTER PROCEDURE dbo.RecoverGraphDatabase
    @BackupFile NVARCHAR(1000),
    @DatabaseName NVARCHAR(100),
    @NewDatabaseName NVARCHAR(100) = NULL,
    @DataFilePath NVARCHAR(1000) = NULL,
    @LogFilePath NVARCHAR(1000) = NULL
AS
BEGIN
    SET NOCOUNT ON;
    
    BEGIN TRY
        -- Validate input parameters
        IF NOT EXISTS (SELECT 1 FROM sys.databases WHERE name = 'msdb')
        BEGIN
            RAISERROR('MSDB database is not accessible.', 16, 1);
            RETURN;
        END

        IF @BackupFile IS NULL OR @DatabaseName IS NULL
        BEGIN
            RAISERROR('Backup file and database name must be specified.', 16, 1);
            RETURN;
        END

        -- Use new database name if specified
        DECLARE @RestoreDatabaseName NVARCHAR(100) = 
            ISNULL(@NewDatabaseName, @DatabaseName);

        -- Default paths if not specified
        DECLARE @DefaultDataPath NVARCHAR(1000),
                @DefaultLogPath NVARCHAR(1000);

        -- Get SQL Server default paths
        EXEC master.dbo.xp_instance_regread 
            N'HKEY_LOCAL_MACHINE',
            N'Software\Microsoft\MSSQLServer\MSSQLServer',
            N'DefaultData',
            @DefaultDataPath OUTPUT;

        EXEC master.dbo.xp_instance_regread 
            N'HKEY_LOCAL_MACHINE',
            N'Software\Microsoft\MSSQLServer\MSSQLServer',
            N'DefaultLog',
            @DefaultLogPath OUTPUT;

        SET @DataFilePath = ISNULL(@DataFilePath, @DefaultDataPath);
        SET @LogFilePath = ISNULL(@LogFilePath, @DefaultLogPath);

        -- Ensure paths end with backslash
        IF RIGHT(@DataFilePath, 1) <> '\'
            SET @DataFilePath = @DataFilePath + '\';
        IF RIGHT(@LogFilePath, 1) <> '\'
            SET @LogFilePath = @LogFilePath + '\';

        -- Verify backup file exists
        DECLARE @FileExists INT;
        EXEC master.dbo.xp_fileexist @BackupFile, @FileExists OUTPUT;
        
        IF @FileExists = 0
        BEGIN
            RAISERROR('Backup file does not exist: %s', 16, 1, @BackupFile);
            RETURN;
        END

        -- Get logical file names from backup
        DECLARE @LogicalDataName NVARCHAR(128),
                @LogicalLogName NVARCHAR(128);

        SELECT TOP 1
            @LogicalDataName = f.logical_name
        FROM msdb.dbo.backupset b
        JOIN msdb.dbo.backupmediafamily m ON b.media_set_id = m.media_set_id
        JOIN msdb.dbo.backupfile f ON b.backup_set_id = f.backup_set_id
        WHERE b.database_name = @DatabaseName
        AND f.file_type = 'D'
        ORDER BY b.backup_start_date DESC;

        SELECT TOP 1
            @LogicalLogName = f.logical_name
        FROM msdb.dbo.backupset b
        JOIN msdb.dbo.backupmediafamily m ON b.media_set_id = m.media_set_id
        JOIN msdb.dbo.backupfile f ON b.backup_set_id = f.backup_set_id
        WHERE b.database_name = @DatabaseName
        AND f.file_type = 'L'
        ORDER BY b.backup_start_date DESC;

        IF @LogicalDataName IS NULL OR @LogicalLogName IS NULL
        BEGIN
            RAISERROR('Could not find logical file names in backup history.', 16, 1);
            RETURN;
        END

        -- Check if target database exists
        IF EXISTS (SELECT 1 FROM sys.databases WHERE name = @RestoreDatabaseName)
        BEGIN
            -- Kill existing connections
            DECLARE @SQL NVARCHAR(MAX) = '';
            SELECT @SQL = @SQL + 
                'KILL ' + CAST(spid AS VARCHAR(10)) + ';'
            FROM master..sysprocesses 
            WHERE dbid = DB_ID(@RestoreDatabaseName)
            AND spid != @@SPID;

            IF LEN(@SQL) > 0
                EXEC(@SQL);
        END

        -- Execute restore
        DECLARE @RestoreSQL NVARCHAR(MAX) = '
        RESTORE DATABASE ' + QUOTENAME(@RestoreDatabaseName) + '
        FROM DISK = ''' + @BackupFile + '''
        WITH 
            MOVE ''' + @LogicalDataName + ''' TO ''' + 
                @DataFilePath + @RestoreDatabaseName + '.mdf'',
            MOVE ''' + @LogicalLogName + ''' TO ''' + 
                @LogFilePath + @RestoreDatabaseName + '_log.ldf'',
            REPLACE,
            STATS = 10;';

        EXEC(@RestoreSQL);

        PRINT 'Database restored successfully to: ' + @RestoreDatabaseName;
    END TRY
    BEGIN CATCH
        DECLARE @ErrorMessage NVARCHAR(4000) = ERROR_MESSAGE(),
                @ErrorSeverity INT = ERROR_SEVERITY(),
                @ErrorState INT = ERROR_STATE();

        RAISERROR(@ErrorMessage, @ErrorSeverity, @ErrorState);
    END CATCH
END;